In [126]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [127]:
from reusable_modules.metrics import MSE, MAE, RMSE, R2_Score
from reusable_modules.gd_poly import  gradient_descent 
from reusable_modules.constants import (
   learning_rates , iteration_counts 
)


In [128]:
df = pd.read_csv('data/zuu crew scores.csv')
df = df[df['CourseName'] == 'Foundations of ML']
df.head(5)

,MemberName,EducationLevel,Attendance,TotalHours,AssignmentsCompleted,HackathonParticipation,GitHubScore,PeerReviewScore,CourseName,CapstoneScore
0,Theekshana Rathnayake,3,79.9,43.7,2,0,62.8,5.0,Foundations of ML,45.3
1,Mayura Sandakalum Sellapperuma,2,76.8,95.6,6,0,87.4,2.7,Foundations of ML,78.8
2,Amila Narangoda,3,96.6,75.9,8,0,98.4,2.8,Foundations of ML,65.4
4,Tharusha Vihanga,2,83.2,24.0,6,0,41.8,4.2,Foundations of ML,40.1
7,Chamath Perera,3,86.5,88.0,5,0,23.9,1.3,Foundations of ML,68.2


Pick Features & Target

In [129]:
FEATURES = ["Attendance", "TotalHours", "AssignmentsCompleted",
            "HackathonParticipation", "PeerReviewScore"]
TARGET = "CapstoneScore"


In [130]:
X_full = df[FEATURES].values
y_full = df[TARGET].values

print (X_full.shape)
print (y_full.shape)


(72, 5)
(72,)


In [131]:
def train_test_split_np(X, y, test_size=0.2, seed=42):
    rng = np.random.default_rng(seed)
    m = X.shape[0]
    idx = np.arange(m); rng.shuffle(idx)
    m_test = int(np.round(m * test_size))
    test_idx = idx[:m_test]; train_idx = idx[m_test:]
    return X[train_idx], X[test_idx], y[train_idx], y[test_idx]

X_train_raw, X_test_raw, y_train, y_test = train_test_split_np(X_full, y_full, test_size=0.2, seed=42)

Scale features

In [132]:
mu = X_train_raw.mean(axis=0)                 # (p,)
sd = X_train_raw.std(axis=0) + 1e-12          # (p,)
X_train = (X_train_raw - mu) / sd
X_test  = (X_test_raw  - mu) / sd

Add an intercept term.

In [133]:
def add_intercept(X):
    return np.c_[np.ones((X.shape[0], 1)), X]

Xtr = add_intercept(X_train)
Xte = add_intercept(X_test)

In [134]:
results = {}   
rows = []      

for ALPHA in learning_rates:
    for N_ITER in iteration_counts:
       
        beta0 = np.zeros(Xtr.shape[1], dtype=float)

        
        cost_history, beta = gradient_descent(Xtr, y_train, beta0, lr=ALPHA, n_iter=N_ITER)
        final_cost = cost_history[-1] if len(cost_history) else np.inf

        
        if not np.isfinite(final_cost):
            print(f"SKIP (diverged): α={ALPHA}, iters={N_ITER}")
            continue


        yhat_tr = Xtr.dot(beta)
        yhat_te = Xte.dot(beta)

        
        results[(ALPHA, N_ITER)] = {"beta": beta, "cost_history": cost_history}

       
        rows.append({
            "α": ALPHA,
            "Iterations": N_ITER,
            "Final Cost (Train)": final_cost,
            "Train MSE":  MSE(y_train, yhat_tr),
            "Train MAE":  MAE(y_train, yhat_tr),
            "Train RMSE": RMSE(y_train, yhat_tr),
            "Train R²":   R2_Score(y_train, yhat_tr),
            "Test MSE":   MSE(y_test,  yhat_te),
            "Test MAE":   MAE(y_test,  yhat_te),
            "Test RMSE":  RMSE(y_test,  yhat_te),
            "Test R²":    R2_Score(y_test,  yhat_te),
        })



Iteration : 0: Cost : 1687.5113
Iteration : 100: Cost : 1684.1549
Iteration : 200: Cost : 1680.8052
Iteration : 300: Cost : 1677.4623
Iteration : 400: Cost : 1674.1260
Iteration : 500: Cost : 1670.7964
Iteration : 600: Cost : 1667.4735
Iteration : 700: Cost : 1664.1572
Iteration : 800: Cost : 1660.8476
Iteration : 900: Cost : 1657.5446
Iteration : 0: Cost : 1687.5113
Iteration : 100: Cost : 1684.1549
Iteration : 200: Cost : 1680.8052
Iteration : 300: Cost : 1677.4623
Iteration : 400: Cost : 1674.1260
Iteration : 500: Cost : 1670.7964
Iteration : 600: Cost : 1667.4735
Iteration : 700: Cost : 1664.1572
Iteration : 800: Cost : 1660.8476
Iteration : 900: Cost : 1657.5446
Iteration : 1000: Cost : 1654.2483
Iteration : 1100: Cost : 1650.9585
Iteration : 1200: Cost : 1647.6753
Iteration : 1300: Cost : 1644.3987
Iteration : 1400: Cost : 1641.1287
Iteration : 1500: Cost : 1637.8652
Iteration : 1600: Cost : 1634.6082
Iteration : 1700: Cost : 1631.3578
Iteration : 1800: Cost : 1628.1138
Iteration

In [135]:
summary_df = (
    pd.DataFrame(rows)
      .sort_values(by=["Test RMSE", "Final Cost (Train)"], ascending=[True, True])
      .reset_index(drop=True)
)
summary_df

,α,Iterations,Final Cost (Train),Train MSE,Train MAE,Train RMSE,Train R²,Test MSE,Test MAE,Test RMSE,Test R²
0,0.00100,10000,8.614573,17.229147,3.362731,4.150801,0.939966,41.585245,5.140081,6.448662,0.849305
1,1.00000,1000,8.614562,17.229123,3.363081,4.150798,0.939966,41.593449,5.141690,6.449298,0.849275
2,1.00000,5000,8.614562,17.229123,3.363081,4.150798,0.939966,41.593449,5.141690,6.449298,0.849275
3,1.00000,10000,8.614562,17.229123,3.363081,4.150798,0.939966,41.593449,5.141690,6.449298,0.849275
4,0.10000,5000,8.614562,17.229123,3.363081,4.150798,0.939966,41.593449,5.141690,6.449298,0.849275
5,0.10000,1000,8.614562,17.229123,3.363081,4.150798,0.939966,41.593449,5.141690,6.449298,0.849275
6,0.10000,10000,8.614562,17.229123,3.363081,4.150798,0.939966,41.593449,5.141690,6.449298,0.849275
7,0.00100,5000,8.701443,17.402886,3.370588,4.171677,0.939360,42.166498,5.108788,6.493574,0.847199
8,0.00100,1000,236.235585,472.471171,20.433088,21.736402,-0.646308,467.145912,19.942849,21.613559,-0.692825
9,0.00001,10000,1383.098618,2766.197236,50.282414,52.594650,-8.638712,2550.233822,48.153764,50.499840,-8.241438
